In [ ]:
%matplotlib inline

from matplotlib.animation import FuncAnimation

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy
import tqdm
import sys

##### Parameters

In [ ]:
# dimension
dim = 1
# diffusion coefficient
D = 2.48e-4
# length
L = 1e-4
# points
P = 1e1
# delta length
dx = L / P
# time span
t_span = (0., 1.)
# delta time
dt = ((1 / P)**2) / (2 * D)
# vesicle transport speed (µm / min)
v = 12.3e-5
# vesicle production rate
gamma = 2e-8
# convertion factor (nutrients to vesicles)
alpha = 2e0
# nutrient used for maintenance
m = 1.8e-3

In [ ]:
# time array
t  = np.arange(*t_span, dt)
# number of time steps
nt = len(t)
# grid points X axis
X = np.linspace(0, L, int(P))
# number of discritized space 
nx = len(X)

#### Vesicules

\begin{equation}
    v\frac{\delta \omega}{\delta x} + \frac{\delta \omega}{\delta t} = \gamma C_{in}
\end{equation}

\begin{equation}
    \frac{\delta \omega}{\delta t} = - v\frac{\delta \omega}{\delta x} + \gamma C_{in}
\end{equation}

\begin{equation}
    \omega_{t}(x, t) = - v \omega_{x}(x, t) + \gamma C_{in}
\end{equation}

\begin{equation}
    \omega_{j}^{i+1} = \omega_{j}^{i} + dt * ( - v \frac{\omega_{j+1}^{i}-\omega_{j}^{i}}{dx} + \gamma C_{in})
\end{equation}

##### Nutrients

\begin{equation}
\label{eq:head_equation_1d}
	\delta_{t}C_{in}(x, t) = D \frac{\delta^{2}C_{in}}{\delta x^{2}}(x, t) + J_{C_{in}}(x,t) - (\alpha \gamma + m) C_{in}
\end{equation}

\begin{equation}
    D2 = \begin{bmatrix}
			b & c & 0 & \cdots & 0\\
			a & b & c & 0 & \vdots\\
			0 & \ddots & \ddots & \ddots & 0\\
			\vdots & 0 & a & b & c\\
			0 & \cdots & 0 & a & b
    \end{bmatrix}
\end{equation}

\begin{equation}
\label{eq:a}
	A = \frac{\alpha \Delta t}{\Delta x^{2}}D2
\end{equation}

\begin{equation}
\label{eq:crank_nicolson_method}
	\frac{T_{i}^{n+1}-T_{i}^n}{\Delta t} = \frac{\alpha}{2} \left(\frac{T_{i-1}^{n+1}-2T_{i}^{n+1}+T_{i+1}^{n+1}}{\Delta x^{2}} + \frac{T_{i-1}^{n}-2T_{i}^{n}+T_{i+1}^{n}}{\Delta x^{2}}\right)
\end{equation}

\begin{equation}
\label{eq:crank_nicolson_method_matrix}
	(I-A_{cn})T^{n+1} = (I + A_{cn})T^{n} \Leftrightarrow T^{n+1} = (I-A_{cn})^{-1}(I + A_{cn})T^{n}
\end{equation}

In [ ]:
def d2_mat_dirichlet(nx, dx):
	diagonals = [[1.], [-2.], [1.]]
	offsets = [-1, 0, 1]
	d2mat = scipy.sparse.diags(diagonals, offsets, shape=(nx-2,nx-2)).toarray()
	return d2mat / dx**2

In [ ]:
I = np.eye(nx - 2)

D2 = d2_mat_dirichlet(nx, dx)

A = alpha * dt * D2
A_cn = 0.5 * A

M1 = np.linalg.inv(I - A_cn)
M2 = (I + A_cn)
M3 = np.dot(M1, M2)

##### Simulation

In [ ]:
indices = np.indices(X.shape)
mask = np.logical_and.reduce(np.ma.masked_inside(indices, indices.min()+1, indices.max()-1).astype(bool).mask)

boundary = np.zeros(X.shape)
source = np.empty(X.shape)

In [ ]:
omega0 = np.zeros(nx)
omega_indices = np.arange(0, nx)
C0 = np.zeros(nx)
C_indices = np.arange(nx, 2*nx)
JC0 = np.zeros(nx)
JC_indices = np.arange(2*nx, 3*nx)
JC0[1:-1] = 1e-6
L0 = np.full((1), dx)
L_indices = np.arange(3*nx, 3*nx+1)

y0 = np.array([*omega0, *C0, *JC0, *L0])
y0

In [ ]:
def system_pde(t, y, *args):
    if len(args):
        n_tanks, = args
    omega = y[omega_indices]
    C = y[C_indices]
    JC = y[JC_indices]
    L = y[L_indices]
    
    res = np.empty(y.shape)

    res[omega_indices[1:-1]] = (-v * omega[omega_indices[:-2]] - omega[omega_indices[2:]]) / dx + gamma * C[1:-1] 
    res[omega_indices[0]] = 0
    res[omega_indices[-1]] = L
    
    res[C_indices[1:-1]] = np.dot(M3, C[1:-1]) + np.dot(M1, (JC[1:-1] - (alpha * gamma + m) * C[1:-1]) * dt)
    res[C_indices] = np.where(mask, res[C_indices], boundary)
    
    res[JC_indices[1:-1]] = -JC[1:-1]
    res[JC_indices] = np.where(mask, res[JC_indices], boundary)
    
    res[L_indices] = 1 / (1 + np.exp(-omega[-1]))
    
    return res

In [ ]:
system_pde(0, y0)

In [ ]:
n_tanks = 3
max_tanks = 100
t = t_span[0]
history = np.array([y0])
l_history = np.array([])

count = 0

def print_sol(y):
    print("shape")
    print(y.shape)
    print("omega")
    print(y[omega_indices])
    print("C")
    print(y[C_indices])
    print("JC")
    print(y[JC_indices])
    print("L")
    print(y[L_indices])
    
print_sol(y0)

with tqdm.tqdm(total=max_tanks) as pbar:
    while n_tanks < max_tanks or t >= t_span[1]:
        sol = scipy.integrate.solve_ivp(system_pde, [t, t + dt], history[-1], args=(n_tanks,))
        
#         print(f"count: {count}")
        print_sol(sol.y[:,-1])
#         print(history[-1][-1])
#         print(sol.y[:,-1][-1])
        if history[-1][-1] == sol.y[:,-1][-1]: 
            break

        nl = n_tanks * dx
        if sol.y[-1][-1] >= nl:
            tmp = np.where(sol.y[-1] >= nl)[0][0] - 1
            history = np.concatenate((history, [sol.y[:,tmp]]))
            t += sol.t[tmp]
            n_tanks += 1
        else:
#             history = np.concatenate((history, [sol.y[:,-1]]))
            t += dt
        pbar.update(1)
        count += 1

In [ ]:
print(n_tanks)
print(history.shape)
print_sol(history[-1])
h_unique = np.unique(history[:,-1])

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)

ax.scatter(np.arange(h_unique.shape[0]), h_unique)
ax.plot(np.arange(h_unique.shape[0]), h_unique)

plt.show()

\begin{equation}
\begin{gathered}
\frac{d \omega_{1}}{d t}=\frac{v}{\Delta x}\left(\omega_{0}-\omega_{1}\right)+\frac{D}{\Delta x^{2}}\left(\omega_{0}-\omega_{1}\right) \\
-\frac{D}{\Delta x^{2}}\left(\omega_{1}-\omega_{2}\right)-\frac{1}{Y_{\phi}} \frac{k_{p} \omega_{1}}{K_{p}+\omega_{1}}-m \rho_{X}
\end{gathered}
\end{equation}


\begin{equation}
\frac{d \phi_{1}}{d t}=-\frac{\psi}{\Delta x}\left(\phi_{1}\right)+\frac{k_{p} \omega_{1}}{K_{P}+\omega_{1}}
\end{equation}

\begin{equation}
\begin{gathered}
\frac{d \omega_{1}}{d t}=\frac{v}{\Delta x}\left(\omega_{0}-\omega_{1}\right)+\frac{D}{\Delta x^{2}}\left(\omega_{0}-\omega_{1}\right) \\
-\frac{D}{\Delta x^{2}}\left(\omega_{1}-\omega_{2}\right) -m \rho_{X}
\end{gathered}
\end{equation}

\begin{equation}
\frac{d \phi_{1}}{d t}=-\frac{\psi}{\Delta x}\left(\phi_{1}\right)
\end{equation}


\begin{equation}
\begin{gathered}
\frac{d \omega_{i}}{d t}=\frac{v}{\Delta x}\left(\omega_{i-1}-\omega_{i}\right)+\frac{D}{\Delta x^{2}}\left(\omega_{i-1}-\omega_{i}\right) \\
-\frac{D}{\Delta x^{2}}\left(\omega_{i}-\omega_{i+1}\right)-\frac{1}{Y_{\phi}} \frac{k_{p} \omega_{i}}{K_{P}+\omega_{i}}-m \rho_{X}
\end{gathered}
\end{equation}

\begin{equation}
\frac{d \phi_{i}}{d t}=\frac{\psi}{\Delta x}\left(\phi_{i-1}-\phi_{i}\right)+\frac{k_{p} \omega_{i}}{K_{p}+\omega_{i}}
\end{equation}

\begin{equation}
\begin{gathered}
\frac{d \omega_{i}}{d t}=\frac{v}{\Delta x}\left(\omega_{i-1}-\omega_{i}\right)+\frac{D}{\Delta x^{2}}\left(\omega_{i-1}-\omega_{i}\right) \\
-\frac{D}{\Delta x^{2}}\left(\omega_{i}-\omega_{i+1}\right)-m \rho_{X}
\end{gathered}
\end{equation}

\begin{equation}
\frac{d \phi_{i}}{d t}=\frac{\psi}{\Delta x}\left(\phi_{i-1}-\phi_{i}\right)
\end{equation}

\begin{equation}
\frac{d L}{d t}=Y_{L} \frac{k_{c} \phi_{n}}{K_{C}+\phi_{n}}
\end{equation}

\begin{equation}
\frac{d \omega_{n}}{d t}=-\left(\frac{\omega_{n}}{L}\right) \frac{d L}{d t}+\frac{v}{L} \omega_{n-1}+\frac{D}{L\left(\frac{L+\Delta x}{2}\right)}\left(\omega_{n-1}-\omega_{n}\right)-m \rho_{X}
\end{equation}

\begin{equation}
\frac{d \phi_{n}}{d t}=-\left(\frac{\phi_{n}}{L}\right) \frac{d L}{d t}+\frac{\psi}{L} \phi_{n-1}-\left(\frac{1}{A L}\right) \frac{k_{c} \phi_{n}}{K_{C}+\phi_{n}}
\end{equation}